In [97]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score

In [98]:
# Read the data
GDP_data = pd.read_excel('dataset\API_NY.GDP.MKTP.CD_DS2_en_excel_v2_4770502.xls', usecols=np.r_[0, 24:65], skiprows=3)
literacy_rate_data = pd.read_excel('dataset\API_SE.ADT.LITR.ZS_DS2_en_excel_v2_4773710.xls', usecols=np.r_[0, 24:65], skiprows=3)
mortality_rate_data = pd.read_excel('dataset\API_SP.DYN.IMRT.IN_DS2_en_excel_v2_4770604.xls', usecols=np.r_[0, 24:65], skiprows=3)
population_data = pd.read_excel('dataset\API_SP.POP.TOTL_DS2_en_excel_v2_4770385.xls', usecols=np.r_[0, 24:65], skiprows=3)
EVI_data = pd.read_excel('dataset\API_TX.VAL.MRCH.XD.WD_DS2_en_excel_v2_4774581.xls', usecols=np.r_[0, 24:65], skiprows=3)

In [99]:
GDP_data.head()

,Country Name,1980,1981,1982,1983,1984,1985,1986,1987,1988,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,4.055866e+08,4.877095e+08,5.966480e+08,...,2.637989e+09,2.615084e+09,2.727933e+09,2.791061e+09,2.963128e+09,2.983799e+09,3.092179e+09,3.202235e+09,3.368970e+09,2.610039e+09
1,Africa Eastern and Southern,1.706561e+11,1.743889e+11,1.672681e+11,1.749199e+11,1.601357e+11,1.362988e+11,1.525199e+11,1.861466e+11,2.041420e+11,...,9.641807e+11,9.725734e+11,9.834729e+11,1.003768e+12,9.245228e+11,8.827213e+11,1.021119e+12,1.007240e+12,1.001017e+12,9.274845e+11
2,Afghanistan,3.641723e+09,3.478788e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.819041e+10,2.020357e+10,2.056449e+10,2.055058e+10,1.999816e+10,1.801956e+10,1.889635e+10,1.841885e+10,1.890449e+10,2.014344e+10
3,Africa Western and Central,1.120313e+11,2.110035e+11,1.871637e+11,1.381152e+11,1.142627e+11,1.165073e+11,1.074975e+11,1.103218e+11,1.089435e+11,...,6.804560e+11,7.360399e+11,8.322169e+11,8.924979e+11,7.669580e+11,6.905454e+11,6.837480e+11,7.663597e+11,7.947191e+11,7.847997e+11
4,Angola,5.930503e+09,5.550483e+09,5.550483e+09,5.784342e+09,6.131475e+09,7.553560e+09,7.072063e+09,8.083872e+09,8.769251e+09,...,1.094366e+11,1.249982e+11,1.334016e+11,1.372444e+11,8.721930e+10,4.984049e+10,6.897277e+10,7.779294e+10,6.930911e+10,5.361907e+10


In [100]:
literacy_rate_data.head()

,Country Name,1980,1981,1982,1983,1984,1985,1986,1987,1988,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97.807419,NaN,97.989998
1,Africa Eastern and Southern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.770672,56.088421,...,65.864540,67.391823,68.445107,68.827972,69.282494,70.059601,69.999451,70.420250,71.574051,71.889908
2,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,31.448851,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Africa Western and Central,NaN,NaN,NaN,NaN,NaN,40.603149,40.878811,41.673779,42.446861,...,51.952728,52.603539,53.144989,54.186619,54.979179,55.564968,56.604252,59.619270,60.178661,60.234989
4,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,66.030113,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
# Provided country
country_name = 'United States'
# Get the list of country names
countries = GDP_data['Country Name'].unique().tolist()

# Create a dataframe to store the data for all countries
df = pd.DataFrame(columns=['Country Name', 'Year', 'GDP', 'Literacy Rate', 'Mortality Rate', 'Population', 'Export Value Index'])
for country in countries:
    # Extract the data for the country from each data set
    GDP = GDP_data.loc[GDP_data['Country Name'] == country].iloc[0, 1:].tolist()
    literacy_rate = literacy_rate_data.loc[literacy_rate_data['Country Name'] == country].iloc[0, 1:].tolist()
    mortality_rate = mortality_rate_data.loc[mortality_rate_data['Country Name'] == country].iloc[0, 1:].tolist()
    population = population_data.loc[population_data['Country Name'] == country].iloc[0, 1:].tolist()
    EVI = EVI_data.loc[EVI_data['Country Name'] == country].iloc[0, 1:].tolist()
    
    # Combine the data into a list of tuples
    data = list(zip([country]*len(GDP), GDP_data.columns[1:], GDP, literacy_rate, mortality_rate, population, EVI))
    
    # Append the data to the dataframe
    # df = df.append(pd.DataFrame(data, columns=df.columns))
    df = pd.concat([df, pd.DataFrame(data, columns=df.columns)])

# Print the data for a specific country (e.g., Canada)
# print(df)
print(df.loc[df['Country Name'] == country_name].head())

    Country Name  Year           GDP  Literacy Rate  Mortality Rate  \
0  United States  1980  2.857307e+12            NaN            12.6   
1  United States  1981  3.207041e+12            NaN            12.1   
2  United States  1982  3.343789e+12            NaN            11.7   
3  United States  1983  3.634038e+12            NaN            11.3   
4  United States  1984  4.037613e+12            NaN            10.9   

    Population  Export Value Index  
0  227225000.0           31.375503  
1  229466000.0           30.850243  
2  231664000.0           29.251602  
3  233792000.0           28.971391  
4  235825000.0           30.830603  


In [102]:
model_data = df.loc[df['Country Name'] == country_name]
model_data.head()

,Country Name,Year,GDP,Literacy Rate,Mortality Rate,Population,Export Value Index
0,United States,1980,2.857307e+12,NaN,12.6,227225000.0,31.375503
1,United States,1981,3.207041e+12,NaN,12.1,229466000.0,30.850243
2,United States,1982,3.343789e+12,NaN,11.7,231664000.0,29.251602
3,United States,1983,3.634038e+12,NaN,11.3,233792000.0,28.971391
4,United States,1984,4.037613e+12,NaN,10.9,235825000.0,30.830603


In [103]:
model_data['Year'] = model_data['Year'].astype(int)
model_GDP = model_data[['Year', 'GDP']]
row_GDP = (model_GDP['Year'] >= 1990) & (model_GDP['Year'] <= 2020)
model_data_GDP = model_GDP.loc[row_GDP, ['GDP']].reset_index(drop=True)
model_data_GDP = model_data_GDP.rename(columns={'GDP': 'GDP in Ten Years'})
model_data_GDP

C:\Users\ajieh\AppData\Local\Temp\ipykernel_23764\1868561611.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data['Year'] = model_data['Year'].astype(int)


,GDP in Ten Years
0,5.963144e+12
1,6.158129e+12
2,6.520327e+12
3,6.858559e+12
4,7.287236e+12
5,7.639749e+12
6,8.073122e+12
7,8.577554e+12
8,9.062818e+12
9,9.631174e+12


In [104]:

model_data = model_data.drop(columns=['GDP'])
row_index = (model_data['Year'] >= 1980) & (model_data['Year'] <= 2010)
columns = ['Year', 'Literacy Rate', 'Population', 'Export Value Index', 'Mortality Rate']
model_data_range = model_data.loc[row_index, columns]
model_data_range

train_data = pd.concat([model_data_range, model_data_GDP], axis=1)
train_data.head()

,Year,Literacy Rate,Population,Export Value Index,Mortality Rate,GDP in Ten Years
0,1980,NaN,227225000.0,31.375503,12.6,5.963144e+12
1,1981,NaN,229466000.0,30.850243,12.1,6.158129e+12
2,1982,NaN,231664000.0,29.251602,11.7,6.520327e+12
3,1983,NaN,233792000.0,28.971391,11.3,6.858559e+12
4,1984,NaN,235825000.0,30.830603,10.9,7.287236e+12


In [105]:
row_index = (model_data['Year'] >= 2011) & (model_data['Year'] <= 2020)
columns = ['Year', 'Literacy Rate', 'Population', 'Export Value Index', 'Mortality Rate']
test_data = model_data.loc[row_index, columns].reset_index(drop=True)
test_data

,Year,Literacy Rate,Population,Export Value Index,Mortality Rate
0,2011,NaN,311583481.0,189.598909,6.1
1,2012,NaN,313877662.0,197.680959,6.0
2,2013,NaN,316059947.0,202.015160,6.0
3,2014,NaN,318386329.0,207.250875,5.9
4,2015,NaN,320738994.0,192.164933,5.8
5,2016,NaN,323071755.0,185.570699,5.7
6,2017,NaN,325122128.0,197.753857,5.7
7,2018,NaN,326838199.0,212.807775,5.6
8,2019,NaN,328329953.0,210.144913,5.5
9,2020,NaN,331501080.0,183.089582,5.4


In [106]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 0 to 30
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Year                31 non-null     int32  
 1   Literacy Rate       0 non-null      float64
 2   Population          31 non-null     float64
 3   Export Value Index  31 non-null     float64
 4   Mortality Rate      31 non-null     float64
 5   GDP in Ten Years    31 non-null     float64
dtypes: float64(5), int32(1)
memory usage: 1.6 KB


In [107]:
train_data.isna().sum()

Year                   0
Literacy Rate         31
Population             0
Export Value Index     0
Mortality Rate         0
GDP in Ten Years       0
dtype: int64

In [108]:
# Fill the missing values in the training set
train_data['Literacy Rate'].fillna((train_data['Literacy Rate'].mean()), inplace=True)
train_data['Export Value Index'].fillna((train_data['Export Value Index'].mean()), inplace=True)
train_data['GDP in Ten Years'].fillna((train_data['GDP in Ten Years'].mean()), inplace=True)
train_data['Mortality Rate'].fillna((train_data['Mortality Rate'].mean()), inplace=True)
train_data['Population'].fillna((train_data['Population'].mean()), inplace=True)
train_data.dropna(axis=1, inplace=True)
train_data.head()

,Year,Population,Export Value Index,Mortality Rate,GDP in Ten Years
0,1980,227225000.0,31.375503,12.6,5.963144e+12
1,1981,229466000.0,30.850243,12.1,6.158129e+12
2,1982,231664000.0,29.251602,11.7,6.520327e+12
3,1983,233792000.0,28.971391,11.3,6.858559e+12
4,1984,235825000.0,30.830603,10.9,7.287236e+12


In [109]:
# Fill the missing values in the test set
test_data['Literacy Rate'].fillna((test_data['Literacy Rate'].mean()), inplace=True)
test_data['Export Value Index'].fillna((test_data['Export Value Index'].mean()), inplace=True)
test_data['Mortality Rate'].fillna((test_data['Mortality Rate'].mean()), inplace=True)
test_data['Population'].fillna((test_data['Population'].mean()), inplace=True)
test_data.dropna(axis=1, inplace=True)
test_data.head()

,Year,Population,Export Value Index,Mortality Rate
0,2011,311583481.0,189.598909,6.1
1,2012,313877662.0,197.680959,6.0
2,2013,316059947.0,202.015160,6.0
3,2014,318386329.0,207.250875,5.9
4,2015,320738994.0,192.164933,5.8


In [110]:
# Drop the target feature from the train data
X = train_data.drop('GDP in Ten Years', axis=1)
y = train_data['GDP in Ten Years']

# Shape and dimension
print("Dimension of X  = {}\nType of X  = {}\n\nDimension of y  = {}\nType of y  = {}".format(X.shape, type(X), y.shape, type(y)))

Dimension of X  = (31, 4)
Type of X  = <class 'pandas.core.frame.DataFrame'>

Dimension of y  = (31,)
Type of y  = <class 'pandas.core.series.Series'>


In [111]:
# dataMap = sns.heatmap(X.corr(), vmin= -1, vmax= 1, annot= True, cmap='BrBG')

In [112]:
# Create a training and a validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=298)
X_train.shape, X_test.shape

((24, 4), (7, 4))

In [113]:
# Train the machine learning model
init_models = { 'Linear Regression': LinearRegression(),
                'Random forest': RandomForestRegressor(random_state=64),
                'Gradient Boosting Regressor': GradientBoostingRegressor(random_state=79),
               }
R2 = []
models_names = []
for i, (key,model) in enumerate(init_models.items()):
    model.fit(X_train, y_train)
    models_names.append(key)
    R2.append(np.mean(cross_val_score(model, X_train, y_train, cv=5)))
models_scores = pd.DataFrame({'Model Name': models_names, 'R2 Score': R2})
models_scores.head()

,Model Name,R2 Score
0,Linear Regression,0.993302
1,Random forest,0.983854
2,Gradient Boosting Regressor,0.972722


In [114]:
# Choose the best model with the highest R2 score
models_scores.sort_values('R2 Score', ascending=False, inplace=True)
best_model = models_scores.iloc[0]

print('Best Model:')
print(best_model)

Best Model:
Model Name    Linear Regression
R2 Score               0.993302
Name: 0, dtype: object


In [115]:
best_model_name = best_model['Model Name']
best_model = init_models[best_model_name]
best_model.fit(X_train, y_train)

LinearRegression()

In [116]:
R2_train = []
R2_test = []
y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)
R2_train.append(r2_score(y_train, y_pred_train))
R2_test.append(r2_score(y_test, y_pred_test))

# Create a dataframe with the R2 scores for each model
models_scores = pd.DataFrame({'Model Name': best_model,'R2 Train': R2_train, 'R2 Test': R2_test})
models_scores

,Model Name,R2 Train,R2 Test
0,LinearRegression(),0.996173,0.995332


In [117]:
y_pred_test

array([1.27460601e+13, 1.01300464e+13, 6.21395132e+12, 1.56352773e+13,
       1.97631146e+13, 9.10954273e+12, 1.06186012e+13])

In [118]:
prediction = best_model.predict(test_data)
prediction

array([2.22987229e+13, 2.28697616e+13, 2.33914144e+13, 2.39165057e+13,
       2.40967054e+13, 2.44202852e+13, 2.50650738e+13, 2.57073535e+13,
       2.60284962e+13, 2.60719893e+13])

In [119]:
# Create a conclusion dataframe and append 'Year' columns
conclusion = pd.DataFrame()
conclusion['Year'] = test_data['Year'] + 10
conclusion['GDP'] = prediction
conclusion

,Year,GDP
0,2021,2.229872e+13
1,2022,2.286976e+13
2,2023,2.339141e+13
3,2024,2.391651e+13
4,2025,2.409671e+13
5,2026,2.442029e+13
6,2027,2.506507e+13
7,2028,2.570735e+13
8,2029,2.602850e+13
9,2030,2.607199e+13


In [120]:
# Save model prediction as a csv file
conclusion.to_csv('prediction.csv', index=False)